In [1]:
#Add repo path to the system path
from pathlib import Path
import os, sys
repo_path= Path.cwd().resolve()
while '.gitignore' not in os.listdir(repo_path): # while not in the root of the repo
    repo_path = repo_path.parent #go up one level
sys.path.insert(0,str(repo_path)) if str(repo_path) not in sys.path else None

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
device = 0

from importlib import import_module
from sklearn.model_selection import KFold
from monai.transforms import (
    Compose,
    ScaleIntensityd,
    EnsureTyped,
    EnsureChannelFirstd,
    Resized,
)
import torchvision
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
from sklearn.metrics import jaccard_score
import SimpleITK as sitk
from PIL import Image
import cv2 as cv

# special imports
from datasets_utils.datasets import ABUS_dataset, ABUS_test
sys.path.append(str(repo_path / 'SAMed')) if str(repo_path / 'SAMed') not in sys.path else None
from SAMed.segment_anything import sam_model_registry

In [10]:
from scipy.ndimage import binary_dilation, binary_erosion
from scipy.ndimage import label, generate_binary_structure


# Morph-Operations 3D

In [6]:
# Load your 3D segmentation mask (assuming it's a binary mask) with sitk
segmentation_mask = sitk.ReadImage(repo_path / 'experiments/inference/segmentation/data/predictions/MASK_110.nii.gz')
segmentation_mask = sitk.GetArrayFromImage(segmentation_mask)
print(f'The size of the segmentation mask is {segmentation_mask.shape}')

The size of the segmentation mask is (353, 682, 865)


In [8]:
structuring_element = np.ones((3, 3, 3), dtype=bool)  # You can adjust the size

# Perform dilation
dilated_mask = binary_dilation(segmentation_mask, structure=structuring_element)
print(f'The shape of the diliated mask is {dilated_mask.shape}')

The shape of the diliated mask is (353, 682, 865)


In [9]:
# save as nii.gz
saving_path = repo_path /  'experiments/inference/segmentation/data/predictions' / f'MASK_110_diluted.nii.gz'
# using sitk
sitk.WriteImage(sitk.GetImageFromArray(segmentation_mask), str(saving_path))

# Conected components

In [17]:
# Load your 3D segmentation mask (assuming it's a binary mask) with sitk
segmentation_mask = sitk.ReadImage(repo_path / 'experiments/inference/segmentation/data/predictions/MASK_110.nii.gz')
segmentation_mask = sitk.GetArrayFromImage(segmentation_mask)
print(f'The size of the segmentation mask is {segmentation_mask.shape}')

# Define the structuring element for connected component analysis
structuring_element = generate_binary_structure(3, 1)  # 3x3x3 connectivity

# Perform connected component labeling
labeled_mask, num_features = label(segmentation_mask, structure=structuring_element)

# Find the size of each connected component
component_sizes = np.bincount(labeled_mask.ravel())

# Identify the label of the largest component (excluding background)
largest_component_label = np.argmax(component_sizes[1:]) + 1

# Create a new mask containing only the largest component
largest_component_mask = labeled_mask == largest_component_label
# transform boolean to int
largest_component_mask = largest_component_mask.astype(np.int8)

# save largest component mask
largest_component_mask = sitk.GetImageFromArray(largest_component_mask)
sitk.WriteImage(largest_component_mask, repo_path / 'experiments/inference/segmentation/data/predictions/MASK_110_largest_component.nii.gz')

The size of the segmentation mask is (353, 682, 865)
